## PART ONE

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mp
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.sandbox.regression.gmm import IV2SLS 

In [ ]:
input_table = pd.read_csv('/Users/dylanbai/Desktop/Predictive Modeling/midterm_partone.csv')
input_table.head()

In [ ]:
model_iv = sm.OLS(input_table["Inventory Turnover"],input_table[["Constant","Current Ratio","Quick Ratio",\
                                                                 "Debt Asset Ratio"]]).fit()
endog_predict = model_iv.predict(input_table[["Constant","Current Ratio","Quick Ratio","Debt Asset Ratio"]])
input_table["Endogenous Param"] = endog_predict

In [ ]:
model_2sls = sm.OLS(input_table["Stock Change"], input_table[["Constant","Endogenous Param",\
                                                              "Operating Profit","Interaction Effect",\
                                                             ]]).fit()
model_2sls.summary()

In [ ]:
y_vals  = np.array(input_table["Stock Change"])
x_vals  = np.array(input_table[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

In [ ]:
class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3, delta = params  # 更新后的参数包括delta
        endog = self.endog
        exog = self.exog
        inst = self.instrument
        
        # 计算新的moment条件
        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] -delta
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] -delta
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] -delta

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1,0.1])
res = gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=4).fit(beta0)

res.summary()



# 计算delta的t统计量
# t_stat_delta = res.params[4] / res.bse[4]
##p_value_delta = 1 - stats.t.cdf(np.abs(t_stat_delta), res.nobs - 1)

# 输出delta的t统计量和p值
##print(f"Delta的t统计量: {t_stat_delta}")
##print(f"Delta的p值: {p_value_delta}")

# 判断行业专家的假设是否成立
##if p_value_delta < 0.05:  # 选择显著性水平为0.05
    #print("行业专家的假设在统计上是成立的（拒绝零假设）。")
#else:
    #print("没有足够的证据来支持行业专家的假设（接受零假设）。")

## PART TWO

In [123]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, roc_curve, roc_auc_score, precision_recall_curve, confusion_matrix, recall_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [124]:
df=pd.read_csv('/Users/dylanbai/Desktop/Predictive Modeling/midterm_parttwo.csv')

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8081 entries, 0 to 8080
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Years of Education after High School  8081 non-null   int64 
 1   Requested Credit Amount               8081 non-null   object
 2   Number of Dependents                  8081 non-null   object
 3   Monthly Income                        8081 non-null   object
 4   Monthly Expense                       8081 non-null   object
 5   Marital Status                        8081 non-null   object
 6   Credit Rating                         8081 non-null   object
dtypes: int64(1), object(6)
memory usage: 442.1+ KB


In [126]:
df.head()

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative


In [127]:
df.describe()

,Years of Education after High School
count,8081.000000
mean,2.608588
std,1.571835
min,0.000000
25%,1.000000
50%,3.000000
75%,3.000000
max,7.000000


In [128]:
df['Number of Dependents'].unique()

array(['No dependent', 'Less than 2', 'More than 2'], dtype=object)

In [142]:
df['Monthly Income'].unique()

array([0, 1, 2, 3, 4])

In [144]:
df['Monthly Expense'].unique()

array([0, 1, 2, 3, 4])

In [138]:
df['Marital Status'].unique()

array([2, 1, 0])

In [137]:
df['Requested Credit Amount'].unique()

array([0, 1, 2])

In [105]:
df['Credit Rating'].unique()

array(['Positive', 'Negative'], dtype=object)

In [106]:
#Dependents = pd.get_dummies(df['Number of Dependents'])
###CreditAmount= pd.get_dummies(df['Requested Credit Amount'])
#Income = pd.get_dummies(df['Monthly Income'])
#Expense=pd.get_dummies(df['Monthly Expense'])
#Marital= pd.get_dummies(df['Marital Status'])


In [143]:
df['Credit Rating'] = df['Credit Rating'].replace({'Positive': 1, 'Negative': 0})
df['Number of Dependents'] = df['Number of Dependents'].replace({'No dependent':0, 'Less than 2':1, 'More than 2':2})
df['Requested Credit Amount'] = df['Requested Credit Amount'].replace({'Low':0, 'Medium':1, 'High':2})
df['Monthly Income'] = df['Monthly Income'].replace({'Very low':0, 'Low':1, 'Moderate':2, 'High':3, 'Very High':4})
df['Monthly Expense'] = df['Monthly Expense'].replace({'Very low':0, 'Low':1, 'Moderate':2, 'High':3, 'Very high':4})
df['Marital Status'] = df['Marital Status'].replace({'Married':2, 'Single':1, 'Not specified':0})


In [108]:
#df_dummies = pd.concat([df,Dependents,CreditAmount,Income,Expense,Marital],axis=1)

In [109]:
#df_dummies.drop(['Number of Dependents','Requested Credit Amount','Monthly Expense','Monthly Income','Marital Status'],axis=1, inplace=True)

In [130]:
df.columns

Index(['Years of Education after High School', 'Requested Credit Amount',
       'Number of Dependents', 'Monthly Income', 'Monthly Expense',
       'Marital Status', 'Credit Rating'],
      dtype='object')

In [145]:
x = df[['Years of Education after High School', 'Requested Credit Amount',
       'Number of Dependents', 'Monthly Income', 'Monthly Expense',
       'Marital Status']]
y = df['Credit Rating']

In [146]:
log = LogisticRegression()

In [147]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.5, random_state=1234)

In [148]:
num = ['Years of Education after High School', 'Requested Credit Amount',
       'Number of Dependents', 'Monthly Income', 'Monthly Expense',
       'Marital Status']

In [149]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num)
    ],
    remainder='passthrough')

# Fit the transformer using the training data
x_train_transformed = preprocessor.fit_transform(x_train)

In [150]:
x_test_transformed = preprocessor.fit_transform(x_test)

In [151]:
log.fit(x_train_transformed, y_train)

LogisticRegression()

In [152]:
pred_log = log.predict(x_test_transformed)

In [153]:
print(f1_score(y_test,pred_log))
print(accuracy_score(y_test,pred_log))
print(recall_score(y_test,pred_log))
print(precision_score(y_test,pred_log))

0.9239781653574758
0.8586983419945559
1.0
0.8586983419945559


In [ ]:
yhat_log